In [2]:
%matplotlib inline
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [79]:
from consts import sensor_day_seconds as day_second
from consts import sensor_idd as idd
from consts import sensor_idds_uniq as idds_uniq
from consts import sensor_idds_uniq_sorted as idds_uniq_sorted
from consts import sensor_a_temperature as a_temperature
from consts import sensor_s_temperature as s_temperature
from consts import sensor_unix_column as unix_column
from consts import sensor_idd as idd
epoch_length = 30

In [81]:
idds_uniq_sorted = sub_sensor_list

In [14]:
input_file = open('dict_list1.pkl','rb')
input_list = pickle.load(input_file)
input_file.close()

In [19]:
input_file = open('dict_list3.pkl','rb')
input_list = pickle.load(input_file)
input_file.close()

In [20]:
# 将epoch epoch_len epoch_time 都flat化
len_list = []
time_stamp_list = []
epoch_list = []
for start_time in input_list:
    # print(len(start_time[0]))
    for dict in start_time[0]:
        len_list.append(len(dict))
        epoch_list.append(dict)
    start_point = start_time[1]
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    time_stamp_list.extend(time_stamps[:-1])
len_array = np.array(len_list)
time_stamp_array = np.array(time_stamp_list) # 时间序列
epoch_array = np.array(epoch_list)

In [94]:
time_stamp_array[indices]

array([1165245058, 1165245388, 1165257148, ..., 1178743378, 1178743408,
       1178744698])

In [87]:
len(epoch_list)

550080

In [86]:
len(ddf_list)

550080

In [5]:
start_list = np.arange(1162393768,1178747998+day_second,day_second)
ddf_list = []
for start_point in start_list:
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    sub_df_list = []
    df = pd.read_hdf('day1.h5',str(start_point))
    for i in range(len(time_stamps)-1):
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] <= time_stamps[i+1])]
        sub_df_list.append(sub_df)
    ddf_list.extend(sub_df_list)

In [21]:
index1 = np.where(len_array > 40)[0]
maybe_array = epoch_array[index1]

In [22]:
# init maybe_dict 
maybe_dict = {}
for i in idds_uniq:
    maybe_dict[i] = 0
print(maybe_dict)


{7: 0, 8: 0, 11: 0, 14: 0, 17: 0, 19: 0, 23: 0, 26: 0, 27: 0, 31: 0, 33: 0, 34: 0, 35: 0, 40: 0, 46: 0, 47: 0, 49: 0, 51: 0, 55: 0, 57: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 65: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 75: 0, 76: 0, 79: 0, 80: 0, 81: 0, 82: 0, 84: 0, 87: 0, 88: 0, 89: 0, 93: 0, 96: 0, 97: 0, 98: 0, 100: 0, 103: 0, 104: 0, 109: 0, 111: 0, 121: 0}


In [23]:
for item in maybe_array:
    uniq_item = np.unique(item)
    for j in uniq_item:
        maybe_dict[j] += 1
print(maybe_dict)

{7: 116368, 8: 311230, 11: 311649, 14: 297530, 17: 311626, 19: 295072, 23: 309388, 26: 309247, 27: 77054, 31: 299208, 33: 267283, 34: 316424, 35: 274661, 40: 266279, 46: 314496, 47: 114937, 49: 308374, 51: 318062, 55: 138159, 57: 224804, 59: 216308, 60: 292967, 61: 231792, 62: 263384, 63: 208200, 65: 294875, 69: 198539, 70: 316076, 71: 203416, 72: 301465, 73: 306707, 75: 303260, 76: 310711, 79: 273899, 80: 295663, 81: 251044, 82: 131717, 84: 308895, 87: 315132, 88: 234990, 89: 283497, 93: 302051, 96: 313535, 97: 310211, 98: 221392, 100: 306061, 103: 318015, 104: 313801, 109: 190240, 111: 303233, 121: 302922}


In [11]:
for item in maybe_array:
    uniq_item = np.unique(item)
    for j in uniq_item:
        maybe_dict[j] += 1
print(maybe_dict)

{7: 122847, 8: 367667, 11: 373104, 14: 363110, 17: 371860, 19: 353906, 23: 366271, 26: 364369, 27: 112280, 31: 359589, 33: 365842, 34: 375659, 35: 375659, 40: 353985, 46: 372432, 47: 121452, 49: 367229, 51: 377511, 55: 368538, 57: 364875, 59: 255396, 60: 349257, 61: 248856, 62: 302648, 63: 264866, 65: 336513, 69: 376076, 70: 375960, 71: 240499, 72: 361416, 73: 368516, 75: 348366, 76: 370370, 79: 325637, 80: 355182, 81: 306862, 82: 168521, 84: 366407, 87: 376175, 88: 303331, 89: 373975, 93: 361673, 96: 373433, 97: 369090, 98: 282533, 100: 364699, 103: 377825, 104: 373251, 109: 224304, 111: 348422, 121: 345429}


In [24]:
maybe_dict_sorted = sorted(maybe_dict.items(),key=lambda kv:(kv[1],kv[0]),reverse=True)
maybe_dict_sorted = np.array(maybe_dict_sorted)

In [78]:
sub_sensor_list = maybe_dict_sorted.T[0]
print(sub_sensor_list)
for item in sub_sensor_list:
    print(item,',')

[ 51 103  34  70  87  46 104  96  11  17   8  76  97  23  26  84  49  73
 100  75 111 121  93  72  31  14  80  19  65  60  89  35  79  33  40  62
  81  88  61  57  98  59  63  71  69 109  55  82   7  47  27]
51 ,
103 ,
34 ,
70 ,
87 ,
46 ,
104 ,
96 ,
11 ,
17 ,
8 ,
76 ,
97 ,
23 ,
26 ,
84 ,
49 ,
73 ,
100 ,
75 ,
111 ,
121 ,
93 ,
72 ,
31 ,
14 ,
80 ,
19 ,
65 ,
60 ,
89 ,
35 ,
79 ,
33 ,
40 ,
62 ,
81 ,
88 ,
61 ,
57 ,
98 ,
59 ,
63 ,
71 ,
69 ,
109 ,
55 ,
82 ,
7 ,
47 ,
27 ,


In [70]:
maybe_dict_sorted.T[1][:40]

array([318062, 318015, 316424, 316076, 315132, 314496, 313801, 313535,
       311649, 311626, 311230, 310711, 310211, 309388, 309247, 308895,
       308374, 306707, 306061, 303260, 303233, 302922, 302051, 301465,
       299208, 297530, 295663, 295072, 294875, 292967, 283497, 274661,
       273899, 267283, 266279, 263384, 251044, 234990, 231792, 224804])

In [26]:
# 测试sensor数量为多少合适
def get_sub_index(sensor_count):
    sub_sensor_list = maybe_dict_sorted.T[0][:sensor_count]
    sub_index = []
    for i, item in enumerate(epoch_array):
        uniq_item = np.unique(item)
        diff = np.setdiff1d(sub_sensor_list,uniq_item)
        if len(diff) == 0:
            sub_index.append(i)
    return len(sub_index),sub_index

In [27]:
# sensor数量与合适epoch之间的关系
sensor_count_list = np.linspace(10,50,41,dtype=int)
sub_index_list = []
for i in sensor_count_list:
    tmp = get_sub_index(i)
    sub_index_list.append(tmp)
    print(i,tmp[0])

10 307126
11 291106
12 268665
13 253685
14 240794
15 232564
16 221419
17 211967
18 201214
19 188411
20 164617
21 158915
22 143072
23 135891
24 123002
25 113508
26 103543
27 95747
28 82678
29 72359
30 68113
31 57984
32 43272
33 37865
34 34475
35 33165
36 22007
37 18107
38 16879
39 11131
40 10886
41 9704
42 8205
43 2999
44 2951
45 2911
46 2857
47 1478
48 400
49 0
50 0


In [28]:
sub_index_array = np.array(sub_index_list)

In [6]:
def get_length_count(window):
    for item in sub_index_array:
        zeros = np.zeros_like(epoch_array)
        zeros[item[1]] = 1
        sum = 0
        for j in range(len(zeros)):
            tmp = zeros[j:j+window]
            if 0 not in tmp:
                sum += 1
        print(sum)

In [29]:
output_file = open('sub_index_array_drop_na.pkl','wb')
pickle.dump(sub_index_array,output_file)
output_file.close()

In [7]:
input_file = open('sub_index_array.pkl','rb')
sub_index_array = pickle.load(input_file)
input_file.close()

In [8]:
get_length_count(7)

148328
132212
112581
98628
83055
70682
60151
53141
46963
38975
32943
27983
24729
21201
18281
16287
13963
12151
10287
8737
7624
6181
5411
4335
3961
3655
3141
2731
2369
1451
1222
1020
948
369
344
147
134
119
106
0
0


In [44]:
def get_data_matrixs(sensor_count,window,ddf_list = None,sub_index_array = None):
    print(idds_uniq_sorted[:sensor_count])
    tmp = sub_index_array[sensor_count-10][1]
    zeros = np.zeros_like(epoch_array)
    zeros[tmp] = 1

    sum_list = []
    for i in range(len(zeros)):
        tmp1 = zeros[i:i+window]
        if 0 not in tmp1:
            sum_list.append(i)
            
    return sum_list
indices = get_data_matrixs(19,7,ddf_list,sub_index_array)

[103, 51, 87, 69, 70, 35, 34, 89, 96, 104, 11, 46, 17, 76, 97, 55, 73, 8, 49]


In [31]:
len(indices)

29898

In [43]:
def drop_useless_sensor(df,sensor_list):
    # print(sensor_list)
    for i in df[idd].values:
        if i not in sensor_list:
            df = df[df[idd] != i]
    # print(len(df))
    if len(df) < len(sensor_list):
        return None
    return df
t = drop_useless_sensor(ddf_list[indices[3]],idds_uniq_sorted[:19])
print(len(t))

19


In [37]:
def handle_df(ddf_list,indice,sensor_list):
    df = ddf_list[indice]
    df = df.drop_duplicates(idd)
    df = drop_useless_sensor(df,sensor_list)
    print(len(df))

handle_df(ddf_list,indices[0],idds_uniq_sorted[:19])

19


In [15]:
t = t.sort_values(by=idd)

In [16]:
t[s_temperature].values

array([10.412, 10.412, 12.35 , 11.6  , 12.725,  8.85 , 10.725, 11.037,
       11.287, 11.037, 10.85 , 10.1  , 10.725, 11.037, 10.6  , 10.35 ,
       10.662, 11.1  ,  9.475])

In [18]:
flat_matrix = np.zeros(100)

In [20]:
indice_dict = {63:10,23:14,17:15,47:20,7:21,27:22,26:24,14:25,121:26,46:27,8:28,111:32,75:33,57:34,59:35,40:36,34:37,61:41,71:42,65:43,82:45,62:46,76:48,51:51,11:52,70:53,49:54,60:57,84:60,81:61,55:62,72:63,73:64,35:66,31:67,69:68,79:69,103:70,80:71,96:72,97:75,100:76,33:77,19:78,93:79,109:80,87:81,104:84,89:91,88:92,98:94}

In [21]:
for i,indice in enumerate(t[idd].values):
    flat_matrix[indice_dict[indice]] = t[s_temperature].values[i]

In [22]:
flat_matrix.reshape(10,10)

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   , 12.35 ,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  8.85 ,
        10.412,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , 11.6  ,
         0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        10.725,  0.   ],
       [ 0.   , 11.037, 10.412, 10.85 , 10.725,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   ,  0.   , 11.287,  0.   , 10.1  ,  0.   , 12.725,  0.   ,
        11.037,  0.   ],
       [11.1  ,  0.   , 10.35 ,  0.   ,  0.   , 10.662,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   , 11.037,  0.   ,  0.   ,  9.475,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ],
       [ 0.   , 10.6  ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ]])

In [23]:
for i in indice_dict:
    flat_matrix[indice_dict[i]] = i

In [24]:
flat_matrix.reshape(10,10)

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 63.,   0.,   0.,   0.,  23.,  17.,   0.,   0.,   0.,   0.],
       [ 47.,   7.,  27.,   0.,  26.,  14., 121.,  46.,   8.,   0.],
       [  0.,   0., 111.,  75.,  57.,  59.,  40.,  34.,   0.,   0.],
       [  0.,  61.,  71.,  65.,   0.,  82.,  62.,   0.,  76.,   0.],
       [  0.,  51.,  11.,  70.,  49.,   0.,   0.,  60.,   0.,   0.],
       [ 84.,  81.,  55.,  72.,  73.,   0.,  35.,  31.,  69.,  79.],
       [103.,  80.,  96.,   0.,   0.,  97., 100.,  33.,  19.,  93.],
       [109.,  87.,   0.,   0., 104.,   0.,   0.,   0.,   0.,   0.],
       [  0.,  89.,  88.,   0.,  98.,   0.,   0.,   0.,   0.,   0.]])

In [62]:
def convert_df_to_dm(ddf_list,indice,sensor_list,window=7):
    # print(sensor_list)
#     print(len(sensor_list))
    df_list = []
    for i in range(window):
        df_list.append(ddf_list[indice+i])
#     df = ddf_list[indice]
    return_list = []
    for df in df_list:
        df = df.drop_duplicates(idd)
#         print(indice,type(df))
#         print(len(df))
        df = drop_useless_sensor(df,sensor_list)
        if type(df) == type(None):
            break
        df = df.sort_values(by=idd)
        flat_matrix = np.zeros(100)
        indice_dict = {63:10,23:14,17:15,47:20,7:21,27:22,26:24,14:25,121:26,46:27,8:28,111:32,75:33,57:34,59:35,40:36,34:37,61:41,71:42,65:43,82:45,62:46,76:48,51:51,11:52,70:53,49:54,60:57,84:60,81:61,55:62,72:63,73:64,35:66,31:67,69:68,79:69,103:70,80:71,96:72,97:75,100:76,33:77,19:78,93:79,109:80,87:81,104:84,89:91,88:92,98:94}
        return_list.append(df[s_temperature].values)
#         for i,indice in enumerate(df[idd].values):
#             flat_matrix[indice_dict[indice]] = df[s_temperature].values[i]
    return np.array(return_list).T
    

res_array = convert_df_to_dm(ddf_list,indices[6],idds_uniq_sorted[:19],7)

In [ ]:
res_array = convert_df_to_dm(ddf_list,[6],idds_uniq_sorted[:19],7)

In [83]:
array_list2 = []
for i in indices:
    array_list2.append(convert_df_to_dm(ddf_list,i,idds_uniq_sorted[:19],7))

In [108]:
sum = 0
new_indices = []
for i,item in enumerate(array_list2):
    if np.isnan(item).any():
        print(i)
#         sum+=1
    else:
        new_indices.append(i)

28681
28682
28683
28684
28690
28691
28692
28720
28768
28769
28770
28771
28772
28842
28851
28852
28853
29068
29069
29143
29144
29233
29234
29261
29304
29305
29330
29404
29405
29406
29407
29408
29409
29410
29449
29450
29451
29452
29453
29454
29455
29557
29558
29559
29560
29561
29604
29605
29606
29640
29641
29665
29673
29674
29794
29795
29882
29883
29884
29885
29886
29887
29888
29890


In [114]:
array_list2[new_indices[-1]]

array([[11.1  , 11.1  , 10.912, 11.037, 10.912, 10.787, 10.912],
       [13.912, 13.787, 13.85 , 13.6  , 13.6  , 13.787, 13.537],
       [11.225, 11.225, 11.35 , 11.35 , 11.35 , 11.412, 11.412],
       [11.85 , 11.912, 11.787, 11.787, 11.725, 11.787, 11.85 ],
       [10.225, 10.35 , 10.35 , 10.1  , 10.1  , 10.037, 10.037],
       [11.6  , 11.662, 11.787, 11.662, 11.662, 11.725, 11.662],
       [12.35 , 12.537, 12.537, 12.412, 12.35 , 12.412, 12.475],
       [10.475, 10.475, 10.475, 10.412, 10.475, 10.475, 10.412],
       [10.287, 10.1  , 10.225, 10.1  , 10.037, 10.1  , 10.162],
       [12.35 , 12.475, 12.162, 12.287, 12.35 , 12.287, 12.225],
       [12.537, 12.662, 12.537, 12.6  , 12.6  , 12.537, 12.6  ],
       [10.537, 10.35 , 10.6  , 10.35 , 10.475, 10.287, 10.412],
       [11.287, 11.287, 11.35 , 11.35 , 11.225, 11.162, 11.287],
       [10.162,  9.975,  9.912, 10.037, 10.1  , 10.037, 10.037],
       [ 9.6  ,  9.6  ,  9.537,  9.662,  9.6  ,  9.6  ,  9.537],
       [ 9.975,  9.912,  

In [112]:
len(array_list2)

29898

In [110]:
new_indices[-1]

29897

In [115]:
with open('tensor2.pkl','wb') as f:
    pickle.dump(array_list2,f)

In [100]:
with open('time_stamp_array.pkl','wb') as f:
    pickle.dump(time_stamp_array,f)

In [32]:
with open('indices_na.pkl','wb') as f:
    pickle.dump(indices,f)

In [100]:
with open('new_indices.pkl','wb') as f:
    pickle.dump(new_indices,f)

In [26]:
df = ddf_list[indices[0]]

In [30]:
idds_uniq_sorted[0]

103

In [28]:
df = df.drop_duplicates(idd)

In [31]:
for i in sub_index_array:
    print(i[0])

346050
335675
318371
306654
284198
268521
257301
246621
235724
226255
215845
206646
198888
189089
177260
171293
161414
154213
143979
132642
123279
114319
104536
93387
83788
80676
71524
67828
62249
45123
41373
37263
33852
16125
14077
8054
7153
7021
4894
0
0


In [ ]:
maybe_dict_sorted.T[0][:40]

In [ ]:
df = pd.read_hdf('day1.h5',str(start_list[1]))

In [12]:
ddf_list[sub_index_array[0][1][1]]

,Station ID,Year,Month,Day,Hour,Minute,Second,Time since the epoch [s],Sequence Number,Ambient Temperature,Surface Temperature,Relative Humidity [%]
70645,8,2006,11,27,14,19,15,1164633555,14611,12.74,11.787,66.492
26185,11,2006,11,27,14,19,19,1164633559,29918,12.55,14.850,72.454
70209,14,2006,11,27,14,19,21,1164633561,11760,12.02,9.725,73.431
66903,17,2006,11,27,14,19,13,1164633553,21043,12.84,13.912,70.046
45042,19,2006,11,27,14,19,0,1164633540,21015,11.98,10.912,73.449
77199,23,2006,11,27,14,19,12,1164633552,23921,12.10,10.725,74.674
78066,26,2006,11,27,14,19,5,1164633545,46311,12.63,11.662,72.311
46764,31,2006,11,27,14,19,13,1164633553,13393,13.03,13.850,72.149
49688,33,2006,11,27,14,19,21,1164633561,25014,12.09,11.537,74.516
63331,34,2006,11,27,14,19,24,1164633564,11564,12.10,11.350,71.142


In [10]:
print(len(ddf_list)*len(ddf_list[0]))

550080


In [ ]:
time_stamps

In [ ]:
print(epoch_length)

In [ ]:
sub_index_array[0][1][4]